In [ ]:
%sql

-- Find how many posts each category had between 2018 and 2022.
-- TASK 5
SELECT PIN.category,
       GEO.timestamp as post_year,
       count(*) as category_count
FROM `0a2f66c3e41f_df_geo` AS GEO
LEFT JOIN `0a2f66c3e41f_df_pin` AS PIN
ON GEO.ind = PIN.ind
WHERE EXTRACT(YEAR FROM timestamp) BETWEEN 2018 AND 2022
GROUP BY PIN.category, GEO.`timestamp`
ORDER BY COUNT(*) DESC;

category,post_year,category_count
mens-fashion,2020-08-28T03:52:47.000+0000,4
diy-and-crafts,2020-04-27T13:34:16.000+0000,1
christmas,2022-05-27T11:30:59.000+0000,1
quotes,2019-09-13T04:50:29.000+0000,1
finance,2021-04-19T17:37:03.000+0000,1
education,2019-12-15T03:51:28.000+0000,1
vehicles,2022-01-01T02:26:50.000+0000,1
diy-and-crafts,2019-09-08T22:53:09.000+0000,1
event-planning,2021-03-20T09:32:44.000+0000,1
christmas,2019-11-03T05:41:59.000+0000,1


In [ ]:
%sql
-- TASK 6
-- Find the user with most followers in each country
-- Step 1: For each country find the user with the most followers.
WITH TopUsers AS (
    SELECT
        geo.country,
        pin.poster_name,
        pin.follower_count,
        user.user_name,
        ROW_NUMBER() OVER (PARTITION BY country ORDER BY follower_count DESC) AS rn
    FROM
        df_geo AS geo
    LEFT JOIN 
        df_pin AS pin ON geo.ind = pin.ind
    LEFT JOIN
        df_user AS user ON user.ind = pin.ind  
    WHERE 
        pin.follower_count IS NOT NULL
)
SELECT
    country,
    poster_name,
    follower_count,
    user_name
FROM
    TopUsers
WHERE
    rn = 1
ORDER BY country;


-- Step 2: Based on the above query, find the country with the user with most followers
WITH TopCountry AS (
    SELECT
        geo.country,
        poster_name,
        follower_count,
        ROW_NUMBER() OVER (PARTITION BY country ORDER BY follower_count DESC) AS rn
    FROM
        df_geo AS geo
    LEFT JOIN 
        df_pin AS pin ON geo.ind = pin.ind
    WHERE 
        pin.follower_count IS NOT NULL
)
SELECT
    country,
    MAX(follower_count) AS max_follower_count
FROM
    TopCountry
GROUP BY
    country
ORDER BY
    max_follower_count DESC
LIMIT 1;




country,poster_name,follower_count,user_name
Angola,CraftGossip.com,502000,David Griffith
Anguilla,"Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog",92000,Corey Andrews
Antarctica (the territory South of 60 deg S),HikenDip,500000,Amanda Adams
Armenia,Michelle {CraftyMorning.com},892000,Andrew Baker
Aruba,TheTrendSpotter,211000,Andrea Alexander
Australia,Cultura Colectiva,1000000,Anne Clayton
Azerbaijan,Style Me Pretty,6000000,Amanda Ball
Bangladesh,Architectural Digest,3000000,Claudia Adams
Barbados,Nicki,28000,Brendan Joseph
Bulgaria,"Living Low Key | Save Money, Make Money, & Frugal Living",26000,Anne Allen


In [ ]:
%sql

-- TASK 7
-- What is the most popular category people post to based on the following age groups:
-- 18-24
-- 25-35
-- 36-50
-- +50

SELECT 
    age_group,
    category,
    COUNT(*) AS category_count
FROM (
    SELECT
        CASE
            WHEN age BETWEEN 18 AND 24 THEN '18-24'
            WHEN age BETWEEN 25 AND 35 THEN '25-35'
            WHEN age BETWEEN 36 AND 50 THEN '36-50'
            WHEN age > 50 THEN '50+'
        END AS age_group,
        category
    FROM df_user
    LEFT JOIN df_pin ON df_user.ind = df_pin.ind
) AS age_category
GROUP BY age_group, category
ORDER BY age_group, category_count DESC


age_group,category,category_count
18-24,mens-fashion,6
18-24,diy-and-crafts,3
18-24,christmas,2
18-24,art,1
18-24,tattoos,1
18-24,home-decor,1
25-35,vehicles,2
25-35,diy-and-crafts,2
25-35,travel,2
25-35,beauty,1


In [ ]:
%sql

-- # TASK 8
-- # Find the median follower count for different age group

SELECT
    CASE
        WHEN age BETWEEN 18 AND 24 THEN '18-24'
        WHEN age BETWEEN 25 AND 35 THEN '25-35'
        WHEN age BETWEEN 36 AND 50 THEN '36-50'
        WHEN age > 50 THEN '+50'
        ELSE 'Unknown'
    END AS age_group,
    percentile(follower_count, 0.5) AS 
    median_follower_count
FROM 
    df_user
LEFT JOIN 
    df_pin 
ON df_user.ind = df_pin.ind
GROUP BY
    CASE
        WHEN age BETWEEN 18 AND 24 THEN '18-24'
        WHEN age BETWEEN 25 AND 35 THEN '25-35'
        WHEN age BETWEEN 36 AND 50 THEN '36-50'
        WHEN age > 50 THEN '+50'
        ELSE 'Unknown'
    END
ORDER BY age_group DESC;


age_group,median_follower_count
36-50,6000.0
25-35,43000.0
18-24,502000.0


In [ ]:
%sql
-- TASK 9
-- Find how many users have joined between 2015 and 2020.
SELECT
      count(*) AS number_users_joined,  
      EXTRACT(YEAR FROM date_joined) AS year_column
FROM 
      df_user
WHERE 
      EXTRACT(YEAR FROM date_joined) BETWEEN 2015 AND 2020
GROUP BY 
      year_column
ORDER BY 
      EXTRACT(YEAR FROM date_joined);

number_users_joined,year_column
15,2015
8,2016
3,2017


In [ ]:
%sql

-- Task 10
-- Find the median follower count of users have joined between 2015 and 2020
SELECT 
    EXTRACT(YEAR FROM df_user.date_joined) AS post_year,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY df_pin.follower_count) AS median_follower_count
FROM 
    df_user
LEFT JOIN 
    df_pin ON df_user.ind = df_pin.ind
WHERE 
    EXTRACT(YEAR FROM df_user.date_joined) BETWEEN 2015 AND 2020
GROUP BY 
    EXTRACT(YEAR FROM df_user.date_joined);



post_year,median_follower_count
2015,87500.0
2016,313000.0
2017,6000.0


In [ ]:
%sql

-- TASK 11
-- Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.

SELECT 
      df_user.age AS age_group,
      EXTRACT(YEAR FROM df_user.date_joined) AS post_year,
      PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY df_pin.follower_count) AS median_follower_count
FROM
    df_user
LEFT JOIN 
    df_pin 
ON 
    df_user.ind = df_pin.ind
WHERE
    EXTRACT(YEAR FROM df_user.date_joined) BETWEEN 2015 AND 2020
GROUP BY 
    EXTRACT(YEAR FROM date_joined),
    df_user.age
ORDER BY 
    EXTRACT(YEAR FROM date_joined),
    df_user.age;

age_group,post_year,median_follower_count
20,2015,1750000.0
21,2015,211000.0
22,2015,892000.0
23,2015,92000.0
25,2015,51000.0
26,2015,110000.0
27,2015,26000.0
30,2015,83000.0
36,2015,12500.0
20,2016,502000.0
